# How to make Excel workbooks the easy way with Python and xlsxwriter

## Aaron Wiegel, PhD, Data Scientist

### PyBay 2018 Lightning Talk

## Synthego, a CRISPR/Cas9 manufacturing biotech startup
![crispr-cas9-sgrna](https://help.synthego.com/hc/article_attachments/360000535107/image__1_.png)

_Courtsey of Synthego_

- Chemistry (RNA synthesis) and Biology (Gene edited cell pools)
- Hardware and Systems Engineering (Robots!)
- Software Engineering (Mostly **_Python_**, some of that...... `JavaScript` thing)
- Data Science and Engineering

## The problem

Head of operations wants high-level KPI metrics to report to exeuctives.

I have the data to generate these, but in a 1500 column 300 MB CSV file generated from our internal Django REST API for manufacturing.

We haven't hired a Data Engineer to build proper infrastructure yet.

I don't want to waste my valuable time dealing with clicking around the horrible Excel GUI to make graphs. (I had enough of that from when I was a grad student. <--- why do you think I learned Python?)

![tps_reports](https://media.giphy.com/media/3owyoUHuSSqDMEzVRu/giphy.gif)

(c) Fox 2018

## The solution: xlsxwriter!

Well-documented open-source Python package that allows for automatic generation of Excel workbooks with graphs and formatting.

In [1]:
import os
import xlsxwriter
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

DEFAULT_PATH = os.path.join(os.path.expanduser('~'), 'side/talks/ExcelReport')
DEFAULT_CSV_FILE = 'manufacturing_data.csv'
DEFAULT_OUTPUT_FILE = 'tps_report.xlsx'
DAYS_AGO = 365

df = pd.read_csv(os.path.join(DEFAULT_PATH, 'manufacturing_data.csv'))
df.head()

,sales_order_id,sales_order_datetime,manufacturing_start_datetime,manufacturing_completed_datetime,shipped_datetime,attempts,pass_rate
0,1,2018-01-07 00:03:00,2018-01-07 04:09:22,2018-01-09 11:38:29,2018-01-10 04:13:02,2,0.500000
1,2,2018-01-07 00:07:57,2018-01-07 03:58:25,2018-01-08 10:39:03,2018-01-09 03:44:39,1,1.000000
2,3,2018-01-07 00:13:41,2018-01-07 06:44:16,2018-01-11 22:30:43,2018-01-12 12:06:08,3,0.333333
3,4,2018-01-07 00:34:54,2018-01-07 05:50:40,2018-01-08 17:26:05,2018-01-09 06:31:29,1,1.000000
4,5,2018-01-07 00:58:50,2018-01-07 06:33:56,2018-01-12 05:32:17,2018-01-12 21:24:42,5,0.200000


In [2]:
class TPSReport:
    # Constants, see below
    INTEGER_COLUMNS = ('sales_orders', 'overproduction')
    PERCENT_COLUMNS = ('pass_rate', 'percent_on_time_shipments')
    
    def __init__(self, data_path=DEFAULT_PATH, input_file=DEFAULT_CSV_FILE, 
                 output_file='tps_report.xlsx', days_ago=DAYS_AGO, date_column='shipped_datetime'):
        
        # Set up file paths
        self.input_path = os.path.join(data_path, input_file)
        self.output_xlsx_path = os.path.join(data_path, output_file)
        
        # Set up filter criteria
        self.days_ago = days_ago
        
        # Load data
        self.input_df = self._load_input_data(date_column)
        
    def _load_input_data(self, date_column):
        """load input csv and do some basic clean up"""
        df = pd.read_csv(self.input_path)
        
        df = self._process_input_df(df, date_column)
        
        return df
        
    def _process_input_df(self, df, date_column):
        """Do some basic filtering of initial data"""
        start_date = datetime.utcnow() - timedelta(days=self.days_ago)
        
        # Convert to datetime, filter dataframe
        df['sales_order_datetime'] = pd.to_datetime(df['sales_order_datetime'])
        df['manufacturing_start_datetime'] = pd.to_datetime(df['manufacturing_start_datetime'])
        df['manufacturing_completed_datetime'] = pd.to_datetime(df['manufacturing_completed_datetime'])
        df['shipped_datetime'] = pd.to_datetime(df['shipped_datetime'])
        
        df = df[df[date_column] >= start_date]
        df.set_index(date_column, inplace=True)
        
        return df
    
    def calculate_metrics(self):
        """process data to create higher level metrics"""
        
        # Calculate timing metrics on individual orders
        self.input_df['manufacturing_lead_time'] = (self.input_df['manufacturing_start_datetime']
                                              - self.input_df['sales_order_datetime']) / np.timedelta64(1, 'h')
        
        self.input_df['manufacturing_time'] = (self.input_df['manufacturing_completed_datetime'] 
                                         - self.input_df['manufacturing_start_datetime']) / np.timedelta64(1, 'h')
        
        self.input_df['shipping_lead_time'] = (self.input_df.index
                                         - self.input_df['manufacturing_completed_datetime']) / np.timedelta64(1, 'h')
        
        self.input_df['turn_around_time'] = (self.input_df.index
                                       - self.input_df['sales_order_datetime']) / np.timedelta64(1, 'D')
        
        self.input_df['on_time_shipment'] = self.input_df['turn_around_time'] <= 6
        
        # Calculate weekly KPIs by resampling on datetime index
        resampler = self.input_df.resample('1w', label='left')
        
        # Count number of sales orders and overproduction
        df = pd.DataFrame(resampler['sales_order_id'].count())
        df.rename(columns={'sales_order_id': 'sales_orders'}, inplace=True)
        
        df['overproduction'] = resampler['attempts'].sum()
        df['pass_rate'] = df['sales_orders'] / df['overproduction']
        
        # Calculate descriptive metrics for process timings
        timing_columns = [column for column in self.input_df.columns if column.endswith('_time')]
        for column in timing_columns:
            df[column+'_mean'] = resampler[column].mean()
            df[column+'_median'] = resampler[column].median()
            df[column+'_90th_percentile'] = resampler[column].quantile(q=0.9)
            
        df['percent_on_time_shipments'] = resampler['on_time_shipment'].mean()
        
        # Rename index, drop time part
        df.index = df.index.strftime('%Y-%m-%d')
        df.index.rename('Week', inplace=True)
        
        self.output_df = df
    
    def format_and_write_report(self, graph_dicts):
        """write data and graphs to separate work sheets"""
        writer = pd.ExcelWriter(self.output_xlsx_path, engine='xlsxwriter')
        
        self._write_summary_stats(writer)
        
        self._write_summary_graphs(writer, graph_dicts)
        
        writer.save()
        
        # Add stuff here to say, upload to s3, tableau sever, whatever
        
    def _write_summary_stats(self, writer, sheet_name='overall KPIs'):
        """write and format Excel worksheet with KPIs""" 
        
        self.output_df.to_excel(writer, sheet_name=sheet_name)
        
        workbook = writer.book
        worksheet = writer.sheets[sheet_name]
        
        # Freeze top row
        # worksheet.freeze_panes(row, col[, top_row, left_col])
        worksheet.freeze_panes(1, 0)
        
        # Add number formats
        percentage = workbook.add_format({'num_format': '0%', 'align': 'right'})
        truncated_decimal = workbook.add_format({'num_format': '0.00', 'align': 'right'})
        integer = workbook.add_format({'num_format': '#,###', 'align': 'center'})
        date = workbook.add_format({'num_format': 'yyyy-mm-dd', 'align': 'left'})
        
        # Format first column from index to use date format
        # worksheet.set_column(first_col, last_col, width, cell_format, options)
        worksheet.set_column(0, 0, len(self.output_df.values[0]), date)
        
        # Get last row number
        last_row = len(self.output_df)
        
        # Format other columns (remember index isn't included in pandas dataframe, but is with Excel)
        for column_index, column in enumerate(self.output_df.columns, start=1):
            if column in self.INTEGER_COLUMNS:
                worksheet.set_column(column_index, column_index, len(column), integer)
                
            elif column in self.PERCENT_COLUMNS:
                # Conditional formatting needs to be set outside other formatting
                # worksheet.conditional_format(first_row, first_col, last_row, last_col, options)
                worksheet.conditional_format(1, column_index, last_row, column_index,
                                             {
                                              'type': '2_color_scale',
                                              'min_type': 'num', 'max_type': 'num',
                                              'min_value': 0.0, 'max_value': 1.0,
                                              'min_color': '#f27285',
                                              'max_color': '#ffffff'
                                             })
                
                worksheet.set_column(column_index, column_index, len(column), percentage)
            else:
                worksheet.set_column(column_index, column_index, len(column), truncated_decimal)
        
    def _write_summary_graphs(self, writer, graph_dicts, sheet_name='KPI graphs'):
        """create graphs from graph dict"""
        
        
        workbook = writer.book
        worksheet = workbook.add_worksheet(sheet_name)
        
        # Create marker to track place of graph
        i = 1
        for graph_dict in graph_dicts:
            # Create new chart object
            chart = self._create_chart(workbook, graph_dict)
            worksheet.insert_chart('A'+str(i), chart)
            i += 15
            
    def _create_chart(self, workbook, graph_dict):
        """create new chart and plot values (y) versus categories (x) with axis label"""
        min_row = 1
        max_row = len(self.output_df)
        data_sheet_name = graph_dict['data_sheet_name']
        
        chart_dict = {'type': graph_dict['chart_type']}
        subtype = graph_dict.get('subtype')
        if subtype is not None:
            chart_dict['subtype'] = subtype
        
        chart = workbook.add_chart(chart_dict)
        
        # Plot each value in series
        for value in graph_dict['values']:
            # Remember index is written to Excel workbook as column 0 but not included in the pandas columns
            value_index = self.output_df.columns.get_loc(value) + 1
            
            chart.add_series({
                # Get name from first row of data sheet
                'name': [data_sheet_name, 0, value_index],
                'categories': [data_sheet_name, min_row, 0, max_row, 0],
                'values': [data_sheet_name, min_row, value_index, max_row, value_index]
            })
        
        chart.set_y_axis({'name': graph_dict['axis_label'], 'major_gridlines': {'visible': False}})
        chart.set_x_axis({'name': graph_dict['category'], 'date_axis': True})
        
        return chart

In [3]:
report = TPSReport()
report.calculate_metrics()

In [4]:
graph_dicts = [
    {
        'data_sheet_name': 'overall KPIs',
        'chart_type': 'line',
        'category': 'Week',
        'values': ('overproduction',),
        'axis_label': 'Overproduction',
    },
    {
        'data_sheet_name': 'overall KPIs',
        'chart_type': 'line',
        'category': 'Week',
        'values': ('turn_around_time_mean',
                   'turn_around_time_median',
                   'turn_around_time_90th_percentile'),
        'axis_label': 'Manufacturing Lead Time',
    },
    {
        'data_sheet_name': 'overall KPIs',
        'chart_type': 'column',
        'subtype': 'stacked',
        'category': 'Week',
        'values': ('manufacturing_lead_time_mean',
                   'manufacturing_time_mean',
                   'shipping_lead_time_mean'),
        'axis_label': 'Process Timing'
    }
]

In [5]:
report.format_and_write_report(graph_dicts)

## Big impact from a small report

- Weekly cronjob provided much better visibility into production trends (along with several other regular more detailed reports)
- Useful stopgap solution as we have been developing more robust and functional alternatives (data warehouse, Tableau)
- Simple, small things can make a big difference

## Thank you!
![Synthego](https://orders.synthego.com/static/images/synthego-logo.svg)

[XlsxWriter](https://xlsxwriter.readthedocs.io/) <--- great documentation!

[https://github.com/aawiegel/talks/tree/master/ExcelReport](https://github.com/aawiegel/talks/tree/master/ExcelReport)

*If you know of a package that does the same thing with Tableau workbooks, please come talk to me immediately!*

We're hiring! 

- Full Stack Software Engineers for various roles (LIMS, Commercial, Computational Biology) using Django (No Biology knowledge required)
- Bioinformatics 

(I promise you will not be asked to automatically generate Excel workbooks.)